# Ejercicio 2: Transfer Learning y Fine-tuning con Monitoreo

## Objetivo
Aplicar transfer learning a un modelo pre-entrenado y realizar fine-tuning monitoreando el proceso con TensorBoard.

## Desarrollo

In [5]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Ruta del dataset descargado
data_dir = tf.keras.utils.get_file(
    origin="https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz",
    fname="flower_photos",
    untar=True
)

# Definimos las rutas para entrenamiento y validación
base_dir = os.path.dirname(data_dir)
train_dir = os.path.join(base_dir, "train")
val_dir = os.path.join(base_dir, "val")

# Creamos carpetas para entrenamiento y validación si no existen
if not os.path.exists(train_dir):
    os.makedirs(train_dir)
if not os.path.exists(val_dir):
    os.makedirs(val_dir)

# Listamos las clases (subcarpetas) en el dataset
classes = os.listdir(data_dir)
print(f"Clases encontradas: {classes}")

# Dividimos las imágenes entre entrenamiento (80%) y validación (20%)
for class_name in classes:
    class_path = os.path.join(data_dir, class_name)
    if os.path.isdir(class_path):  # Aseguramos que sea una carpeta
        images = [img for img in os.listdir(class_path) if os.path.isfile(os.path.join(class_path, img))]  # Solo archivos
        train_images, val_images = train_test_split(images, test_size=0.2, random_state=42)
        
        # Movemos imágenes de entrenamiento
        train_class_dir = os.path.join(train_dir, class_name)
        if not os.path.exists(train_class_dir):
            os.makedirs(train_class_dir)
        for img in train_images:
            shutil.copy(os.path.join(class_path, img), os.path.join(train_class_dir, img))
        
        # Movemos imágenes de validación
        val_class_dir = os.path.join(val_dir, class_name)
        if not os.path.exists(val_class_dir):
            os.makedirs(val_class_dir)
        for img in val_images:
            shutil.copy(os.path.join(class_path, img), os.path.join(val_class_dir, img))

print("División completada.")
print(f"Datos de entrenamiento almacenados en: {train_dir}")
print(f"Datos de validación almacenados en: {val_dir}")

Clases encontradas: ['flower_photos']


ValueError: With n_samples=1, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [1]:
# Importamos las librerías necesarias
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import TensorBoard
import datetime
import os


In [2]:
# Configuración de TensorBoard
log_dir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1, write_graph=True)


In [3]:
# Cargamos el modelo pre-entrenado ResNet50 sin las capas superiores
base_model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False)

# Congelamos las capas base
base_model.trainable = False

# Añadimos nuevas capas para fine-tuning
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(2, activation='softmax')(x)  # Asumiendo clasificación binaria

model = Model(inputs=base_model.input, outputs=predictions)

# Compilamos el modelo
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


In [ ]:
# Preprocesamiento de datos
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

val_datagen = ImageDataGenerator(rescale=1.0/255.0)

# Cargamos los datos (ajusta las rutas a tus datasets)
train_generator = train_datagen.flow_from_directory(
    'ruta_a_tus_datos_de_entrenamiento',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')

val_generator = val_datagen.flow_from_directory(
    'ruta_a_tus_datos_de_validacion',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')

In [ ]:
# Entrenamos el modelo
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size,
    epochs=10,
    callbacks=[tensorboard_callback]
)

In [ ]:
# Fine-Tuning: Descongelamos las capas superiores del modelo base
base_model.trainable = True

# Recompilamos el modelo con un learning rate más bajo
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Continuamos el entrenamiento
history_fine_tune = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size,
    epochs=10,
    callbacks=[tensorboard_callback]
)

## Monitoreo con TensorBoard
Abre una terminal y ejecuta el siguiente comando para visualizar los resultados en TensorBoard:

```bash
tensorboard --logdir logs
```

Luego, abre el navegador en la dirección indicada (generalmente http://localhost:6006).